# Dataton BC 2018

**Grupo:** The kernel trick

**Fecha:** 29-10-2018

# Script # 2: Topic modelling a partir de las columnas de texto de la tabla de personas

En este script se hace un análisis no supervisado del texto de la tabla de transacciones. El texto analizado se compone de los campos 'ref1', 'ref2', y 'descripcion'. Estas tres columnas son juntadas, y el texto resultante es pre-procesado para realizar un análisis de *topic modelling*. El propósito es encontrar categorías en los datos a partir del texto, y categorizar los registros de acuerdo a esto.

Se aplica el modelo de Latent Dirichlet Allocation[1] (LDA) para hallar temas en los textos del dataset. Se probó con 20 temas, para tratar de tener un buen número de potenciales categorías. Estos 20 temas fueron luego estudiados de acuerdo a las palabras que los componen, y fueron re-agrupados y nombrados.

El objetivo con esto es crear una categorización de los registros. Luego, esta categorización será utilizada como variable objetivo en un problema supervisado multivariable.

[1] Referencia: https://www.analyticsvidhya.com/blog/2016/08/beginners-guide-to-topic-modeling-in-python/

_____________________________________________________________________________________________________________________________


In [1]:
# Importar librerías
import pickle
import re
from datetime import datetime

import gensim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from gensim import corpora
from nltk.corpus import stopwords

%matplotlib inline

## 1. Unificar y limpiar el texto en una nueva columna de la tabla personas

In [2]:
# Cargar la tabla de personas, tal y como salió del script # 1
personas = pd.read_csv('personas_procesada.csv')
personas.head()

,id_trn_ach,id_cliente,valor_trx,ref1,ref2,sector,subsector,descripcion,date,dia_trx
0,241307506,19,701067.98,Pago de la Planilla Cesantias,CEDULA DE CIUDADANIA,SERVICIOS FINANCIEROS,OTROS SERVICIOS FINANCIEROS,Otras actividades auxiliares de las actividade...,2017-02-10 14:00:00,Friday
1,359125394,35,246964.14,CPV,NaN,SERVICIOS FINANCIEROS,BANCOS,Bancos comerciales,2018-08-08 12:00:00,Wednesday
2,285847659,40,192491.73,Pago Factura Asociado,BancoomevaPFA,SERVICIOS FINANCIEROS,BANCOS,Bancos comerciales,2017-10-03 15:00:00,Tuesday
3,319249942,45,49520.98,Presentación y Pago del Impuesto Predial Unifi...,AAAXDFT null,GOBIERNO,MUNICIPIOS,Actividades ejecutivas de la administración pú...,2018-03-05 11:00:00,Monday
4,335337578,45,431146.54,EDIF MIRABELL PH BOG,NaN,SERVICIOS FINANCIEROS,BANCOS,Bancos comerciales,2018-05-07 14:00:00,Monday


In [5]:
# Conformar una columna de texto, esta es la columna a la cual se le aplica topic modelling.
# Se unen las columnas 'ref1', 'ref2' y 'descripcion'
personas[['ref1','ref2']] = personas[['ref1','ref2']].fillna(value='')
personas['texto'] = personas.ref1 + ' ' + personas.ref2 + ' ' + personas.descripcion

# Se probó utilizando en el texto los campos 'sector' y 'subsector', pero no dio tan buenos resultados
# personas['texto'] = personas.ref1 + ' ' + personas.ref2 + ' ' + personas.sector + ' ' + personas.subsector + ' ' + personas.descripcion


In [6]:
# Para explorar un poco, se imprime el número de valores únicos de las variables categóricas y de texto

for colu in ['ref1','ref2','sector','subsector','descripcion','texto']:
    print('............................................')
    print(colu)
    print(len(pd.unique(personas[colu])))

# Se quitan las columnas 'ref1' y 'ref2', que ya se encuentran embebidas en la columna 'texto'
personas = personas.drop(['ref1','ref2'], 1)
    

............................................
ref1
27452
............................................
ref2
27439
............................................
sector
10
............................................
subsector
54
............................................
descripcion
149
............................................
texto
57185


In [7]:
# Función para hacer una limpieza a los textos para prepararlos para el posterior análisis

def basic_clean(text):
# Texto a minúsculas
    text = text.lower()
# Quita caracteres especiales de las palabras. Solo deja pasar letras y espacio
    text = re.sub(r"[^a-zA-ZñÑáéíóúÁÉÍÓÚ ]"," ", text)
# Reemplaza espacios múltiples por un solo espacio
    text = re.sub(r" +"," ", text)
# Quitar espacios, tabs y enters en los extremos del texto
    text = text.strip(' \t\n\r')
# Quitar stopwords y palabras de menos de 3 letras
    stop = set(stopwords.words('spanish'))
    text = " ".join([i for i in text.split() if (i not in stop) and (len(i) > 2)])
    return text


In [8]:
# Aplicar la función de limpieza a la columna 'texto'
personas.texto = personas.texto.map(basic_clean)

In [9]:
personas.head()

,id_trn_ach,id_cliente,valor_trx,sector,subsector,descripcion,date,dia_trx,texto
0,241307506,19,701067.98,SERVICIOS FINANCIEROS,OTROS SERVICIOS FINANCIEROS,Otras actividades auxiliares de las actividade...,2017-02-10 14:00:00,Friday,pago planilla cesantias cedula ciudadania acti...
1,359125394,35,246964.14,SERVICIOS FINANCIEROS,BANCOS,Bancos comerciales,2018-08-08 12:00:00,Wednesday,cpv bancos comerciales
2,285847659,40,192491.73,SERVICIOS FINANCIEROS,BANCOS,Bancos comerciales,2017-10-03 15:00:00,Tuesday,pago factura asociado bancoomevapfa bancos com...
3,319249942,45,49520.98,GOBIERNO,MUNICIPIOS,Actividades ejecutivas de la administración pú...,2018-03-05 11:00:00,Monday,presentación pago impuesto predial unificado a...
4,335337578,45,431146.54,SERVICIOS FINANCIEROS,BANCOS,Bancos comerciales,2018-05-07 14:00:00,Monday,edif mirabell bog bancos comerciales


## 2. Preparar el texto y crear el diccionario (vocabulario) y el corpus (colección de documentos) para entrenar el modelo no supervisado

In [10]:
# Una vez limpio, cada texto se parte para formar una lista de palabras
doc_clean = list(personas.texto)
doc_clean = [doc.split() for doc in doc_clean] 

# Se guarda este objeto
pickle.dump(doc_clean, open('documentos_limpios.pkl', 'wb'))

In [13]:
# Se crea el diccionario a partir de todos los textos, y luego se aplica bag of words (bow) para vectorizar los textos
# y formar el corpus
dictionary = corpora.Dictionary(doc_clean)
corpus = [dictionary.doc2bow(doc) for doc in doc_clean]

# Se guardan estos dos objetos
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

## 3. Entrenamiento del modelo LDA

In [105]:
# Primero se probó este modelo, pero es muy lento para crear el modelo.

# Lda = gensim.models.ldamodel.LdaModel
# ldamodel = Lda(corpus, num_topics=20, id2word = dictionary, passes=10,chunksize=int(0.2*len(personas)),random_state=13,update_every=5)


# Se procedió entonces a probar con la versió que puede paralelizar el entrenamiento en varios núcleos
Lda = gensim.models.ldamulticore.LdaMulticore
ldamodel = Lda(corpus, num_topics=20, id2word = dictionary, passes=10,chunksize=int(0.2*len(personas)),
               random_state=13,workers=3)

# Se guarda el modelo
pickle.dump(ldamodel, open('modelo_lda_2.pkl', 'wb'))

### 3.1 Temas encontrados en el dataset

A continuación se muestran los 20 temas que el algoritmo encontró en el corpus. Para cada tema se muestran las 10 palabras más relevantes, y en la última columna se muestra la categoría que le hemos decidido dar a cada tema, dependiendo de sus palabras.

In [102]:
temas_transacciones = pd.read_excel('Temas de las transacciones.xlsx')
temas_transacciones

,Tema #,Palabra 1,Palabra 2,Palabra 3,Palabra 4,Palabra 5,Palabra 6,Palabra 7,Palabra 8,Palabra 9,Palabra 10,Segmento propuesto
0,0,"0.172*""servicio""","0.166*""pago""","0.162*""servicios""","0.160*""telefonía""","0.160*""fija""","0.131*""movil""","0.027*""fijo""","0.002*""nit""","0.002*""saldo""","0.002*""actividades""",Tecnología y comunicaciones
1,1,"0.102*""educación""","0.063*""pago""","0.052*""técnica""","0.049*""compra""","0.040*""documentos""","0.036*""profesional""","0.035*""universidad""","0.033*""establecimientos""","0.033*""niveles""","0.033*""combinan""",Educación
2,2,"0.197*""actividades""","0.120*""servicio""","0.065*""financiero""","0.052*""apoyo""","0.043*""empresas""","0.037*""jad""","0.027*""cuota""","0.019*""facturas""","0.016*""secuencia""","0.015*""pago""",Otros
3,3,"0.144*""energía""","0.144*""eléctrica""","0.144*""generación""","0.142*""esp""","0.142*""medellin""","0.141*""empresas""","0.141*""publicas""","0.002*""nit""","0.000*""instituciones""","0.000*""tecnológicas""",Servicios públicos
4,4,"0.085*""certificados""","0.083*""actividades""","0.083*""idc""","0.083*""administración""","0.083*""central""","0.083*""pública""","0.083*""ejecutivas""","0.083*""gobierno""","0.083*""transaccion""","0.083*""libertad""",Trámites/gobierno
5,5,"0.256*""bancos""","0.256*""comerciales""","0.136*""cpv""","0.113*""pse""","0.113*""recarga""","0.112*""nequi""","0.002*""colpatria""","0.002*""prepagada""","0.002*""medicina""","0.001*""turismo""",Ahorro y giros
6,6,"0.083*""idc""","0.074*""actividades""","0.064*""central""","0.064*""administración""","0.064*""ejecutivas""","0.064*""pública""","0.064*""gobierno""","0.058*""recarga""","0.057*""cuenta""","0.056*""certificados""",Trámites/gobierno
7,7,"0.201*""servicios""","0.198*""telefonía""","0.198*""fija""","0.177*""pago""","0.176*""saldo""","0.022*""recarga""","0.015*""express""","0.006*""prepago""","0.001*""celular""","0.001*""uso""",Tecnología y comunicaciones
8,8,"0.111*""actividades""","0.083*""pago""","0.060*""administración""","0.059*""ejecutivas""","0.055*""impuesto""","0.054*""pública""","0.054*""municipios""","0.051*""presentación""","0.051*""null""","0.048*""vehiculos""",Pago de impuestos
9,9,"0.166*""pago""","0.166*""actividades""","0.166*""inalámbricas""","0.166*""telecomunicaciones""","0.159*""referencia""","0.141*""express""","0.017*""app""","0.007*""factura""","0.007*""mitigo""","0.000*""nit""",Tecnología y comunicaciones


### 3.2 Asignar los temas a cada registro

Utilizando el modelo LDA obtenido, se asigna un tema, o categoría, a cada registro del dataset. 

Para que un texto pertenezca a un tema en particular, se definió que más del 50% del texto debe pertenecer a este tema. En caso de que ninguno de los temas cumpla esto para un texto en particular, dicho texto se clasifica dentro de 'otros'.

In [103]:
# Función para asignar uno de los temas a un nuevo documento

docu = 'ahorro consignación de dinero'
def definir_tema(docu,dictionary,min_prob=0.5):
    # Se asume que el documento ya está "limpio" (es decir, ya pasó por la función 'basic_clean')
    new_doc = docu.split()
    new_doc_bow = dictionary.doc2bow(new_doc)
    topics = ldamodel.get_document_topics(new_doc_bow)
    tema = topics[np.argmax([i[1] for i in topics])]
    if tema[1] > min_prob:
        return tema[0]
    else:
        return np.nan


In [60]:
personas['tema'] = [definir_tema(docu,dictionary) for docu in personas.texto]

In [72]:
# Aplicar las categorías seleccionadas a los datos del dataset

dict_temas = {0: 'Tecnología y comunicaciones', 1: 'Educación', 2: 'Otros', 3: 'Servicios públicos',
              4: r'Trámites/gobierno', 5: 'Ahorro y giros', 6: r'Trámites/gobierno', 7: 'Tecnología y comunicaciones',
              8: 'Pago de impuestos', 9: 'Tecnología y comunicaciones', 10: 'Otros', 11: r'Transporte/construcción',
              12: 'Pago de impuestos', 13: 'Seguridad social', 14: 'Vivienda', 15: 'Tecnología y comunicaciones',
              16: 'Vivienda', 17: 'Pago de deudas', 18: 'Tarjetas de crédito', 19: r'Trámites/gobierno'}

personas['cat_tema'] = personas.tema.map(dict_temas)
personas['cat_tema'][pd.isna(personas['cat_tema'])] = 'Otros'


C:\Users\edescobar\AppData\Local\Continuum\miniconda3\envs\anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [106]:
# Estudiar la distribución de las categorías de temas en el dataset

print('-------Cantidad de registros por tema -------')
print(personas.cat_tema.value_counts(dropna=False))

print('\n -------Porcentaje de registros por tema -------')
print(personas.cat_tema.value_counts(dropna=False)/len(personas))

-------Cantidad de registros por tema -------
Tecnología y comunicaciones    1173847
Ahorro y giros                  518496
Servicios públicos              442492
Trámites/gobierno               331192
Pago de deudas                  167285
Tarjetas de crédito             157251
Transporte/construcción         144209
Pago de impuestos               139006
Vivienda                         91663
Educación                        53170
Seguridad social                 52319
Otros                            36455
Name: cat_tema, dtype: int64

 -------Porcentaje de registros por tema -------
Tecnología y comunicaciones    0.354917
Ahorro y giros                 0.156769
Servicios públicos             0.133789
Trámites/gobierno              0.100137
Pago de deudas                 0.050579
Tarjetas de crédito            0.047545
Transporte/construcción        0.043602
Pago de impuestos              0.042029
Vivienda                       0.027715
Educación                      0.016076
Segurid

## 4. Crear base para modelo supervisado

Ya se tienen todos los registros del dataset categorizados en los temas encontrados. Estos temas van a ser utilizados como etiquetas en un problema supervisado, en el cual se va a utilizar información del pagador y de la transacción (sin tener en cuenta sector, subsector, referencias y descripción= para tratar de predecir a qué categoría pertenece una transacción.


### 4.1 Balancear categorías

Se trata de buscar un equilibrio entre: tener una representación uniforme de todas las posibles categorías que puede tomar una transacción, y tener un buen número de registros con los cuales entrenar un modelo supervisado.

Basado en la distribución de registros encontrada en la sección **3.2**, se decide tratar de tener 50.000 registros de cada categoría en la base de modelamiento. Hay 1 categoría que no alcanza a tener este número de registros, pero en general la base queda bastante bien balanceada, y con un buen número de registros (**586.455**).

In [2]:
base_modelamiento = pd.DataFrame()
for tema in pd.unique(personas.cat_tema):
    temp = personas.loc[personas.cat_tema == tema].copy()
    n_samps = np.min([50000,len(temp)])
    temp = temp.sample(n=n_samps, random_state=13)
    base_modelamiento = base_modelamiento.append(temp, ignore_index=True)

base_modelamiento.cat_tema.value_counts()

Vivienda                       50000
Tarjetas de crédito            50000
Trámites/gobierno              50000
Pago de deudas                 50000
Educación                      50000
Tecnología y comunicaciones    50000
Seguridad social               50000
Ahorro y giros                 50000
Pago de impuestos              50000
Servicios públicos             50000
Transporte/construcción        50000
Otros                          36455
Name: cat_tema, dtype: int64

In [3]:
# Se hace una tabla cruzada para ver cómo se relacionan las columnas "cat_tema" y "sector". Se puede ver
# que en algunos casos hay unos sectores muy fuertemente relacionados a algunos de los temas encontrados.

pd.crosstab(base_modelamiento.cat_tema, base_modelamiento.sector)

sector,AGROINDUSTRIA,COMERCIO,CONSTRUCCION,GOBIERNO,MANUFACTURA INSUMOS,MEDIOS DE COMUNICACION,PERSONAS,RECURSOS NATURALES,SERVICIOS FINANCIEROS,SERVICIOS NO FINANCIEROS
cat_tema,,,,,,,,,,
Ahorro y giros,0,0,0,0,0,0,0,0,49931,69
Educación,548,192,0,36417,311,1275,0,0,4867,6390
Otros,18,304,512,2822,282,249,5157,2,17970,9139
Pago de deudas,0,0,1483,19,0,1,23,0,48474,0
Pago de impuestos,0,0,0,35432,1,0,0,0,3272,11295
Seguridad social,0,0,1,49984,0,0,0,0,13,2
Servicios públicos,0,0,0,0,0,0,0,49998,0,2
Tarjetas de crédito,0,0,0,0,0,0,0,0,50000,0
Tecnología y comunicaciones,0,83,0,0,0,49916,0,0,0,1


### 4.2 Calcular nuevas caracterísicas y dejar solamente las crts que van al modelo

Ya se había determinado el día de la semana en el que ocurre cada transacción. Ahora, de manera similar, se encuentra el mes, el día del mes y la hora en la que tiene lugar la transacción. Todas estas características van a entrar al modelo supervisado.

Por otro lado, se eliminan de la base de modelamiento las variables que no se inlcuirán en el modelo (sector, subsector, descripción y texto).


In [4]:
base_modelamiento['date'] = pd.to_datetime(base_modelamiento['date'])

base_modelamiento['dia_del_mes_trx'] = [i.day for i in base_modelamiento['date']]
base_modelamiento['mes_trx'] = [i.month_name() for i in base_modelamiento['date']]
base_modelamiento['hora_trx'] = [i.hour for i in base_modelamiento['date']]


In [5]:
# Seleccionar solo las columnas que se van a utilizar en el modelo (más la id de la trx)
base_modelamiento = base_modelamiento[['id_cliente','id_trn_ach','valor_trx','dia_trx','mes_trx','dia_del_mes_trx',
                                      'hora_trx','cat_tema']]


### 4.3 Cruzar con información de pagadores

Finalmente, a la base de modelamiento se le cruzan las variables de los pagadores, para incluir esas crts en el modelo. Las dos tablas se cruzan por medio de la columna 'id_cliente'.

In [6]:
# Cargar la tabla de pagadores, tal y como salió del script # 1
pagadores = pd.read_csv('pagadores_procesada.csv')
pagadores.head()

,id_cliente,seg_str,ocupacion,tipo_vivienda,nivel_academico,estado_civil,genero,edad,ingreso_rango
0,18,PERSONAL PLUS,5,O,U,M,M,78.0,e. (4.4 5.5MM]
1,32,PERSONAL PLUS,E,F,T,M,M,78.0,i. (8.7 Inf)
2,41,EMPRENDEDOR,3,O,I,W,M,78.0,b. (1.1 2.2MM]
3,47,EMPRENDEDOR,7,NaN,I,I,M,78.0,c. (2.2 3.3MM]
4,71,PERSONAL,5,O,S,M,M,78.0,e. (4.4 5.5MM]


In [7]:
base_modelamiento = base_modelamiento.merge(pagadores,left_on='id_cliente', right_on='id_cliente', how='left')
base_modelamiento.head()

,id_cliente,id_trn_ach,valor_trx,dia_trx,mes_trx,dia_del_mes_trx,hora_trx,cat_tema,seg_str,ocupacion,tipo_vivienda,nivel_academico,estado_civil,genero,edad,ingreso_rango
0,116890,326250526,3120.20,Monday,April,2,21,Pago de impuestos,PERSONAL PLUS,1,NaN,S,I,F,32.0,f. (5.5 6.6MM]
1,162625,326254647,118031.07,Monday,April,2,21,Pago de impuestos,PERSONAL PLUS,E,F,U,S,M,37.0,h. (7.6 8.7MM]
2,104683,248482707,17711.09,Wednesday,March,22,11,Pago de impuestos,PERSONAL,1,F,U,M,F,43.0,b. (1.1 2.2MM]
3,189406,286116816,4957.62,Wednesday,October,4,14,Pago de impuestos,EMPRENDEDOR,NaN,NaN,NaN,NaN,NaN,NaN,i. (8.7 Inf)
4,62915,334420397,313244.24,Thursday,May,3,20,Pago de impuestos,PERSONAL PLUS,5,O,U,M,F,62.0,i. (8.7 Inf)


In [8]:
# Guardar la base para modelar y la base con todos los registros y sus respectivos temas
base_modelamiento.to_csv('base_modelo.csv',index=False)
personas.to_csv('personas_temas.csv',index=False)